In [1]:

%xmode Minimal

#  Setting display options 

import pandas as pd
   #
pd.set_option("display.width", 480)

#  Sets horizontal scroll for wide outputs
#
from IPython.display import display, HTML
display(HTML(""))

from tabulate import tabulate

print("--")


Exception reporting mode: Minimal


--


#  Setup stuff: Connectivity

In [2]:

from katana import remote
from katana.remote import import_data

my_client = remote.Client()

print(my_client)


In [4]:

print(remote.Client().client_version)
print(remote.Client().server_version)


0.9.0+20230322T224759Z.73f024927.dev
0.9.0+20230322T224759Z.73f024927.dev


In [5]:

NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph"

print("--")


--


In [6]:

#  CONNECT TO GRAPH

my_graph, *_ = my_client.get_database(name=DB_NAME).find_graphs_by_name(GRAPH_NAME)

print(my_graph)


<_Graph my_graph, 91yr3NQDRHFD9AnV5ZD97EWc6AaR4cKttZegZ8vwy4gQ, 0>


# Louvain algorithm

In [7]:

from katana.remote import analytics


l_cntr = 20                                                #  Used to generate a unique column name
   #
print("--")


--


In [7]:

#  We use this counter to generate a unique column name below 
#
l_cntr += 1
   #
l_output_propname     = "louvain_" + str(l_cntr).zfill(4)


l_result = analytics.louvain_clustering(
   my_graph,
      #
   result_property_name = l_output_propname,
      #
   is_symmetric         = True
   )

print(l_result)
   #
print("--")


          0/? [?op/s]

None
--


# Checking that result above


In [ ]:

#  View the graph
#

l_result = my_graph.query(
   """
   MATCH (n) - [r] -> (m)
   RETURN n, r, m
   """ ,
   contextualize=True,
   )
l_result.view()


          0/? [?op/s]

In [ ]:

#  Unfiltered node data
#

l_query = """

   MATCH (n)
   RETURN n.{0}, ID(n), n
   ORDER BY n.{0} DESC
    
    """.format(l_output_propname)

l_result = my_graph.query_unpaginated(l_query)
   #
print(tabulate(l_result, headers='keys', tablefmt='psql'))

#  Sample data, after adding Narita and Seoul below-
#
#     +----+----------------------+-----------------+-----------------+-------------+------------------+------------------+----------------------+----------+-----------+
#     |    |       n.louvain_0023 |           ID(n) |   n.internal_id | n.labels    | n.airport_code   | n.airport_name   |       n.louvain_0023 | n.type   | n.LABEL   |
#     |----+----------------------+-----------------+-----------------+-------------+------------------+------------------+----------------------+----------+-----------|
#     |  0 | 18446744073709551615 |               2 |               2 | ['Airport'] | NRT              | Narita, Japan    | 18446744073709551615 | node     | nan       |
#     |  1 | 18446744073709551615 |               0 |               0 | ['Airport'] | ORD              | Chicago O-Hare   | 18446744073709551615 | node     | Airport   |
#     |  2 | 18446744073709551615 |               1 |               1 | ['Airport'] | SJC              | San Jose         | 18446744073709551615 | node     | Airport   |
#     |  3 | 18446744073709551615 | 281474976710656 | 281474976710656 | ['Airport'] | ICN              | Seoul, Korea     | 18446744073709551615 | node     | nan       |
#     |  4 | 18446744073709551615 | 562949953421312 | 562949953421312 | ['Airport'] | MKE              | Milwaukee        | 18446744073709551615 | node     | Airport   |
#     |  5 | 18446744073709551615 | 562949953421313 | 562949953421313 | ['Airport'] | DEN              | Denver           | 18446744073709551615 | node     | Airport   |
#     +----+----------------------+-----------------+-----------------+-------------+------------------+------------------+----------------------+----------+-----------+


In [ ]:

#  View output
#

l_query = """

   MATCH (n)
   RETURN n.{0}, ID(n), n.airport_name
   ORDER BY n.{0} DESC
   LIMIT 10
    
    """.format(l_output_propname)

l_result = my_graph.query_unpaginated(l_query)
   #    
print(tabulate(l_result, headers='keys', tablefmt='psql'))


In [ ]:

#  Add a new community
#

l_query = """
   CREATE ( n: Airport { airport_code: 'NRT' } )               //  Notice single curly braces
   SET n.airport_name = 'Narita, Japan' 
   """
      #
display(l_result = my_graph.query(l_query))

l_query = """
   CREATE ( n: Airport { airport_code: 'ICN' } ) 
   SET n.airport_name = 'Seoul, Korea' 
   """
      #
display(l_result = my_graph.query(l_query))


l_query = """
   MATCH
      (n: Airport   ),
      (m: Airport   )
   WHERE n.airport_code     = 'NRT' AND m.airport_code  = 'ICN'
   CREATE (n) -[r: FLIES_TO { DISTANCE: 752, NUM_HOPS: 1}]-> (m)
   """
      #
display(l_result = my_graph.query(l_query))



In [ ]:
#  Go up, run Louvain again ..